# IndoXTC - Fine-tune Full Toxic [XLM-R] (Simpler)
Exploring Indonesian hate speech/abusive & sentiment text classification using multilingual language model.

This kernel is a part of my undergraduate final year project.
Checkout the full github repository:
https://github.com/ilhamfp/indonesian-text-classification-multilingual

In [1]:
#############################
# Experiment configurations #
#############################

INDO_DATA_NAME = 'toxic'
DATA_PATH_INDO = '../input/simpler-preprocess-indonesian-hate-abusive-text'

FOREIGN_DATA_NAME = 'toxic'
DATA_PATH_FOREIGN = '../input/jigsaw-multilingual-toxic-comment-classification'

MODEL_NAME = 'XLM_R'

EXPERIMENT_TYPE = 'C' # A / B / C
TOTAL_DATA = 11852 # 500 / 1000 / 2500 / 5000 / 7500 / 11852
FOREIGN_LANG_DATA_MULT = 3 # 0.5 / 1 / 1.5 / 2 / 3
RANDOM_SEED = 1
VALIDATION_DATA = 0.1
EPOCHS = 19
LEARNING_RATE = 5e-6
USE_TPU = True

In [2]:
from model_full import set_seed, regular_encode, build_model, callback
from load_data import load_experiment_dataset
import os
import random
import pandas as pd
import numpy as np
import torch
import tensorflow as tf
import transformers
from transformers import TFAutoModel, AutoTokenizer

set_seed(seed=RANDOM_SEED)

## TPU Configs

In [3]:
if USE_TPU:
    # Detect hardware, return appropriate distribution strategy
    try:
        # TPU detection. No parameters necessary if TPU_NAME environment variable is
        # set: this is always the case on Kaggle.
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
    else:
        # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
        strategy = tf.distribute.get_strategy()

    print("REPLICAS: ", strategy.num_replicas_in_sync)
    BATCH_SIZE = 8 * strategy.num_replicas_in_sync
    
else:
    BATCH_SIZE = 8 * 8

AUTO = tf.data.experimental.AUTOTUNE
MAX_LEN = 512

if MODEL_NAME == 'XLM_R':
    MODEL = 'jplu/tf-xlm-roberta-large'
elif MODEL_NAME == 'mBERT':
    MODEL = 'bert-base-multilingual-cased'

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


## Load Data

In [4]:
(x_train, y_train), (x_valid, y_valid), (x_test, y_test) = load_experiment_dataset(INDO_DATA_NAME,
                                                                                   FOREIGN_DATA_NAME,
                                                                                   tipe=EXPERIMENT_TYPE, 
                                                                                   total_data=TOTAL_DATA, 
                                                                                   foreign_mult=FOREIGN_LANG_DATA_MULT, 
                                                                                   valid_size=VALIDATION_DATA)

~~~Train Data~~~
Shape:  (11852, 2)
                                                text  label
0   cebong selalu tidak pindah on masuk saja kema...      1
1  sungguh rezim jokowi lebih sadis dari kekuasaa...      1

Label:
1    6578
0    5274
Name: label, dtype: int64

~~~Test Data~~~
Shape:  (1317, 2)
                                                text  label
0  gubernur daerah khusus ibukota salat jumat ber...      0
1   hindu tidak mengenal hari persidangan hindu p...      0
2                                     jancuk jancuk       1
3                bukti on bukti jangan congor doang       1

Label:
1    731
0    586
Name: label, dtype: int64
~~~Data~~~
Shape:  (304222, 2)
                                  text  label
0  haha you guys are a bunch of losers      1
1                  ur a sh tty comment      1

Label:
1    152111
0    152111
Name: label, dtype: int64
~~~Train Data~~~
Shape:  (11852, 2)
                                                text  label
0   cebong selalu ti

## Tokenizing

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [6]:
%%time 

x_train = regular_encode(x_train, tokenizer, maxlen=MAX_LEN)
x_valid = regular_encode(x_valid, tokenizer, maxlen=MAX_LEN)
x_test = regular_encode(x_test, tokenizer, maxlen=MAX_LEN)

CPU times: user 24.9 s, sys: 111 ms, total: 25 s
Wall time: 25 s


## Build Datasets Objects

In [7]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(len(x_train),
             seed=RANDOM_SEED)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

## Load Model

In [8]:
%%time
if USE_TPU:
    with strategy.scope():
        transformer_layer = TFAutoModel.from_pretrained(MODEL)
        model = build_model(transformer_layer, max_len=MAX_LEN, learning_rate=LEARNING_RATE)
        
else:
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=MAX_LEN, learning_rate=LEARNING_RATE)
    
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 512)]             0         
_________________________________________________________________
tf_roberta_model (TFRobertaM ((None, 512, 1024), (None 559890432 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1024)]            0         
_________________________________________________________________
dropout_74 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1025      
Total params: 559,891,457
Trainable params: 559,891,457
Non-trainable params: 0
_________________________________________________________________
CPU times: user 1min 47s, sys: 36.6 s, total: 2min 24s
Wall time: 2min 25s


## Train

In [9]:
n_steps = x_train.shape[0] // BATCH_SIZE
print(n_steps)

666


In [10]:
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    callbacks = callback(), 
    epochs=EPOCHS
)

Train for 666 steps, validate for 19 steps
Epoch 1/19


/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:430: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256002048 elements. This may consume a large amount of memory.
  num_elements)
/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:430: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256002048 elements. This may consume a large amount of memory.
  num_elements)


666/666 [==============================] - 676s 1s/step - loss: 0.5970 - f1: 0.6541 - accuracy: 0.6933 - val_loss: 0.4159 - val_f1: 0.7898 - val_accuracy: 0.8084
Epoch 2/19
666/666 [==============================] - 462s 694ms/step - loss: 0.3375 - f1: 0.8429 - accuracy: 0.8557 - val_loss: 0.3667 - val_f1: 0.8307 - val_accuracy: 0.8489
Epoch 3/19
666/666 [==============================] - 463s 695ms/step - loss: 0.2729 - f1: 0.8765 - accuracy: 0.8886 - val_loss: 0.3499 - val_f1: 0.8665 - val_accuracy: 0.8667
Epoch 4/19
666/666 [==============================] - 462s 694ms/step - loss: 0.2310 - f1: 0.8952 - accuracy: 0.9059 - val_loss: 0.3050 - val_f1: 0.8771 - val_accuracy: 0.8776
Epoch 5/19
666/666 [==============================] - 463s 696ms/step - loss: 0.1903 - f1: 0.9153 - accuracy: 0.9249 - val_loss: 0.2791 - val_f1: 0.8914 - val_accuracy: 0.8962
Epoch 6/19
666/666 [==============================] - 457s 686ms/step - loss: 0.1560 - f1: 0.9312 - accuracy: 0.9393 - val_loss: 0.295

## Save

In [11]:
model.save_weights('model.h5') 

In [12]:
result = pd.DataFrame()
result['y_pred'] = model.predict(test_dataset, verbose=1).flatten()
result['y_true'] = y_test
result.to_csv('result_{}_{}_{}_{}_{}_{}_full.csv'.format(INDO_DATA_NAME,
                                                    FOREIGN_DATA_NAME,
                                                    MODEL_NAME,
                                                    EXPERIMENT_TYPE,
                                                    TOTAL_DATA,
                                                    FOREIGN_LANG_DATA_MULT),
              index=False)

21/21 [==============================] - 49s 2s/step
